# Import Libraries

In [ ]:
import torch
import textattack

from transformers import AutoModelForSequenceClassification, AutoTokenizer

from textattack import Attacker, AttackArgs
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import FasterGeneticAlgorithmJia2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_results import SuccessfulAttackResult, FailedAttackResult

import warnings
warnings.filterwarnings("ignore")

# Load Data

In [ ]:
dataset = HuggingFaceDataset("imdb", split="test")

# Model

In [ ]:
model_name = "textattack/bert-base-uncased-imdb"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Faster Alzantot Genetic Algorithm

In [ ]:
attack = FasterGeneticAlgorithmJia2019.build(model_wrapper)

In [ ]:
attack_args = AttackArgs(
    num_examples=100,
    query_budget=10,
    log_to_csv="faga_results.csv",
    checkpoint_interval=5,
    checkpoint_dir="checkpoints",
    disable_stdout=False,
    random_seed=42
)

In [ ]:
attacker = Attacker(attack, dataset, attack_args)

# Results

In [ ]:
results = attacker.attack_dataset()

In [ ]:
succeeded = 0
failed = 0

for result in results:
    if type(result) == SuccessfulAttackResult:
        succeeded += 1
    elif type(result) == FailedAttackResult:
        failed += 1

In [ ]:
print(f"Succeded: {succeeded}")
print(f"Failed: {failed}")
print(f"Success Rate: {(succeeded / (succeeded + failed)) * 100:.2f}%")